#### Setup and parameters

In [1]:
import warnings
warnings.simplefilter('ignore')
import deeplabcut
import pandas as pd
import ruamel.yaml
import sys
import os
import tensorflow as tf
import numpy as np
import re

wd=r"Z:\lab\NSF forelimb project\Phil_lab\dlc-data-swap"
possum_megavids=[r"Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\11Apr18.LaiRegnault.SEP101.LS.biceps_teres_lat\c1_11Apr.mp4",
               r"Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\11Apr18.LaiRegnault.SEP101.LS.biceps_teres_lat\c2_11Apr.mp4"]
experimenter="Phil"

#### Create new project and extract frames

In [2]:
task='possum101_11Apr'
path_config_file=deeplabcut.create_new_project(task,experimenter,possum_megavids, working_directory=wd, videotype='.mp4', copy_videos=False) 
%matplotlib inline
deeplabcut.extract_frames(path_config_file, userfeedback=False) 

Created "\\rcstore02.rc.fas.harvard.edu\spierce_lab\lab\NSF forelimb project\Phil_lab\dlc-data-swap\possum101_11Apr-Phil-2020-04-13\videos"
Created "\\rcstore02.rc.fas.harvard.edu\spierce_lab\lab\NSF forelimb project\Phil_lab\dlc-data-swap\possum101_11Apr-Phil-2020-04-13\labeled-data"
Created "\\rcstore02.rc.fas.harvard.edu\spierce_lab\lab\NSF forelimb project\Phil_lab\dlc-data-swap\possum101_11Apr-Phil-2020-04-13\training-datasets"
Created "\\rcstore02.rc.fas.harvard.edu\spierce_lab\lab\NSF forelimb project\Phil_lab\dlc-data-swap\possum101_11Apr-Phil-2020-04-13\dlc-models"
Creating the symbolic link of the video
Created the symlink of Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\11Apr18.LaiRegnault.SEP101.LS.biceps_teres_lat\c1_11Apr.mp4 to \\rcstore02.rc.fas.harvard.edu\spierce_lab\lab\NSF forelimb project\Phil_lab\dlc-data-swap\possum101_11Apr-Phil-2020-04-13\videos\c1_11Apr.mp4
Created the symlink of Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\11Apr18.LaiRegnault.SEP101.LS.bicep

In [2]:
import os
import re
        
def scanDir(directory, extension='avi', filters=[], filter_out=True, verbose=False):
    file_list=[]
    for root, dirs, files in os.walk(directory):
        for name in files:
            if name.lower().endswith(extension):
                filename = os.path.join(root, name)
                if verbose == True:
                    print("Found file with extension ."+ extension + ": " + filename)
                file_list.append(filename)
                continue
            else:
                continue
    if len(filters) != 0:
        if filter_out==True:
            for string in filters:
                file_list = [file for file in file_list if not re.search(string, file)]
        else:
            for string in filters:
                file_list = [file for file in file_list if re.search(string, file)]
    return(file_list)

In [3]:
import os
import numpy as np
import cv2
        
def vidToPngs(video_path, output_dir=None, indices_to_match=[], name_from_folder=True):
    frame_index = 0
    frame_counter = 0
    if name_from_folder:
        out_name = os.path.splitext(os.path.basename(video_path))[0]
    else:
        out_name = 'img'
    if output_dir==None:
        out_dir = os.path.join(os.path.dirname(video_path),out_name)
    else:
        out_dir = output_dir
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    frame_rate = round(cap.get(5),2)
    while(cap.isOpened()):
        ret, frame = cap.read()
        frame_counter += 1
        if ret == True:
            if indices_to_match and not frame_index in indices_to_match:
                frame_index += 1
                continue
            else:
                png_name = out_name+str(frame_index).zfill(4)+'.png'
                png_path = os.path.join(out_dir, png_name)
                cv2.imshow('frame',frame)
                cv2.imwrite(png_path, frame)
                frame_counter = 0                
                if cv2.waitKey(1) & 0xFF == ord('q'):
                        break
                frame_index += 1
        else: 
            break
                
    cap.release()
    cv2.destroyAllWindows()
    print("done!")

In [8]:
import os

def matchFrames(extracted_dir):
    extracted_files = scanDir(extracted_dir, extension='png') 
    extracted_indices = [int(os.path.splitext(os.path.basename(png))[0][3:].lstrip('0')) for png in extracted_files]
    unique_indices = []
    unique_indices = [index for index in extracted_indices if not index in unique_indices]
    result = sorted(unique_indices)
    return result   

def extractMatchedFrames(indices, output_dir = None, src_vids=[]):
    extracted_frames = []
    for video in src_vids:
        out_name = os.path.splitext(os.path.basename(video))[0]+'_matched'
        if output_dir is not None:
            output = output_dir
        else:
            output = os.path.join(os.path.dirname(video),out_name)
        vidToPngs(video, output, indices_to_match=indices, name_from_folder=False)
        extracted_frames.append(output)
    return extracted_frames


In [5]:
import pandas as pd
import os

def spliceXma2Dlc(substitute_video,project_path,csv_path, frame_indices):
    substitute_name = os.path.splitext(os.path.basename(substitute_video))[0]
    substitute_data_relpath = os.path.join("labeled-data",substitute_name)
    substitute_data_abspath = os.path.join(project_path,substitute_data_relpath)

    df=pd.read_csv(csv_path,sep=',',header=0,dtype='float',na_values='NaN')
    names = df.columns.values
    parts = [name.rsplit('_',1)[0] for name in names]
    parts_unique = []
    for part in parts:
        if not part in parts_unique:
            parts_unique.append(part)
    df['frame_index']=[os.path.join(substitute_data_relpath,'img'+str(index).zfill(4)+'.png') for index in frame_indices]
    df['scorer']=experimenter
    df = df.melt(id_vars=['frame_index','scorer'])
    new = df['variable'].str.rsplit("_",n=1,expand=True)
    df['variable'],df['coords'] = new[0], new[1]
    df=df.rename(columns={'variable':'bodyparts'})
    df['coords']=df['coords'].str.rstrip(" ").str.lower()
    cat_type = pd.api.types.CategoricalDtype(categories=parts_unique,ordered=True)
    df['bodyparts']=df['bodyparts'].str.lstrip(" ").astype(cat_type)
    newdf = df.pivot_table(columns=['scorer', 'bodyparts', 'coords'],index='frame_index',values='value',aggfunc='first',dropna=False)
    newdf.index.name=None
    ##export
    if not os.path.exists(substitute_data_abspath):
        os.mkdir(substitute_data_abspath)
    data_name = os.path.join(substitute_data_abspath,("CollectedData_"+experimenter+".h5"))
    newdf.to_hdf(data_name, 'df_with_missing', format='table', mode='w')
    newdf.to_csv(data_name.split('.h5')[0]+'.csv')
    print("saved "+str(data_name))
    return substitute_data_abspath, parts_unique

In [6]:
%%capture
import ruamel.yaml


def updateConfig(path_config_file, videos=[],bodyparts=[],corner2move2=512):
    config = ruamel.yaml.load(open(path_config_file))
    if videos:
        video_sets={video:{"crop":"0, 1024, 0, 1024"} for video in videos}
        config['video_sets']=video_sets
    if bodyparts:
        config['bodyparts']=bodyparts
    config['corner2move2']=[corner2move2,corner2move2]
    ruamel.yaml.round_trip_dump(config, sys.stdout)
    with open(path_config_file, 'w') as fp:
        ruamel.yaml.round_trip_dump(config, fp)
        fp.close()
    return path_config_file


# Digitize extracted frames in xmalab, then take marker IDs from output csv and convert digitized data to h5

In [9]:
path_config_file = r"Z:\lab\NSF forelimb project\Phil_lab\dlc-data-swap\possum101_11Apr-Phil-2020-04-13-mult\config.yaml"
substitute_video = r"Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\11Apr18.LaiRegnault.SEP101.LS.biceps_teres_lat\11Apr_mult.mp4"



xma_csv_path = r"Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\11Apr18.LaiRegnault.SEP101.LS.biceps_teres_lat\c1and2_11Apr_matched_xmalab2D.csv"
project_path = os.path.dirname(path_config_file)
extracted_dir = os.path.join(project_path,"labeled-data")




##get matching frames from merged video 
frame_indices = matchFrames(extracted_dir)
spliced_folder, bodyparts = spliceXma2Dlc(substitute_video,project_path,xma_csv_path,frame_indices)

extractMatchedFrames(frame_indices, output_dir = spliced_folder, src_vids=[substitute_video])

##then remove separate cameras from project and insert new folder with merged video and extracted frames
updateConfig(path_config_file, videos=[substitute_video],bodyparts=bodyparts)

saved Z:\lab\NSF forelimb project\Phil_lab\dlc-data-swap\possum101_11Apr-Phil-2020-04-13-mult\labeled-data\11Apr_mult\CollectedData_Phil.h5
done!
Task: possum101_11Apr
scorer: Phil
date: Apr13
project_path: \\rcstore02.rc.fas.harvard.edu\spierce_lab\lab\NSF forelimb project\Phil_lab\dlc-data-swap\possum101_11Apr-Phil-2020-04-13-mult
video_sets:
  Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\11Apr18.LaiRegnault.SEP101.LS.biceps_teres_lat\11Apr_mult.mp4:
    crop: 0, 1024, 0, 1024
bodyparts:
- Body_ds1_crn_cam1
- Body_ds1_crn_cam2
- Body_ds2_int_cam1
- Body_ds2_int_cam2
- Body_ds3_cdl_cam1
- Body_ds3_cdl_cam2
- Body_vn1_crn_cam1
- Body_vn1_crn_cam2
- Body_vn2_int_cam1
- Body_vn2_int_cam2
- Body_vn3_cdl_cam1
- Body_vn3_cdl_cam2
- Scapula_acr_cam1
- Scapula_acr_cam2
- Scapula_spi_cam1
- Scapula_spi_cam2
- Scapula_vtb_cam1
- Scapula_vtb_cam2
- Humerus_dpc_cam1
- Humerus_dpc_cam2
- Humerus_ent_cam1
- Humerus_ent_cam2
- Humerus_ect_cam1
- Humerus_ect_cam2
- Ulna_olc_cam1
- Ulna_olc_cam2
- Ul

C:\Users\Phil\.conda\envs\dlc-windowsGPU\lib\site-packages\ipykernel_launcher.py:5: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  """


'Z:\\lab\\NSF forelimb project\\Phil_lab\\dlc-data-swap\\possum101_11Apr-Phil-2020-04-13-mult\\config.yaml'

#### Check imported labels

In [53]:
deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

Creating images with labels by Phil.
They are stored in the following folder: \\rcstore02.rc.fas.harvard.edu\spierce_lab\lab\NSF forelimb project\Phil_lab\dlc-data-swap\possum101_11Apr-Phil-2020-04-13\labeled-data\11Apr_diff_labeled.
If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


#### 8. Create training set

In [3]:
deeplabcut.create_training_dataset(r"C:\Users\Phil\Development\DeepLabCut\dev\possum101_11Apr-Phil-2020-04-13-diff\config.yaml")

C:\Users\Phil\Development\DeepLabCut\dev\possum101_11Apr-Phil-2020-04-13-diff\training-datasets\iteration-0\UnaugmentedDataSet_possum101_11AprApr13  already exists!
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


#### 9. Start training

In [ ]:
deeplabcut.train_network(r"C:\Users\Phil\Development\DeepLabCut\dev\possum101_11Apr-Phil-2020-04-13-diff\config.yaml", displayiters=50,saveiters=10000, maxiters=200000)

Config:
{'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16],
                [17],
                [18],
                [19],
                [20],
                [21],
                [22],
                [23],
                [24],
                [25],
                [26],
                [27],
                [28],
                [29],
                [30],
                [31],
                [32],
                [33],
                [34],
                [35],
                [36],
                [37],
                [38],
                [39],
                [40],
                [41],
                [42],
                [43]],
 'all_joints_names': ['Body_ds1_c

Switching batchsize to 1, as default/tensorpack/deterministic loaders do not support batches >1. Use imgaug loader.
Starting with standard pose-dataset loader.
Initializing ResNet
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Loading ImageNet-pretrained resnet_50
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\Phil\.conda\envs\dlc-windowsGPU\lib\site-packages\deeplabcut\pose_estimation_tensorflow\models\pretrained\resnet_v1_50.ckpt
Max_iters overwritten as 200000
Display_iters overwritten as 50
Save_iters overwritten as 10000
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'weigh_only_present_joints': False, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': 'C:\\Users\\Phil\\Development\\DeepLabCut\\dev\\possum101_11Apr-Phil-2020-

iteration: 50 loss: 0.1823 lr: 0.005
iteration: 100 loss: 0.0292 lr: 0.005
iteration: 150 loss: 0.0259 lr: 0.005
iteration: 200 loss: 0.0243 lr: 0.005
iteration: 250 loss: 0.0268 lr: 0.005
iteration: 300 loss: 0.0254 lr: 0.005
iteration: 350 loss: 0.0246 lr: 0.005
iteration: 400 loss: 0.0236 lr: 0.005
iteration: 450 loss: 0.0249 lr: 0.005
iteration: 500 loss: 0.0260 lr: 0.005
iteration: 550 loss: 0.0261 lr: 0.005
iteration: 600 loss: 0.0261 lr: 0.005
iteration: 650 loss: 0.0237 lr: 0.005
iteration: 700 loss: 0.0250 lr: 0.005
iteration: 750 loss: 0.0248 lr: 0.005
iteration: 800 loss: 0.0256 lr: 0.005
iteration: 850 loss: 0.0253 lr: 0.005
iteration: 900 loss: 0.0239 lr: 0.005
iteration: 950 loss: 0.0234 lr: 0.005
iteration: 1000 loss: 0.0232 lr: 0.005
iteration: 1050 loss: 0.0228 lr: 0.005
iteration: 1100 loss: 0.0224 lr: 0.005
iteration: 1150 loss: 0.0213 lr: 0.005
iteration: 1200 loss: 0.0230 lr: 0.005
iteration: 1250 loss: 0.0204 lr: 0.005
iteration: 1300 loss: 0.0203 lr: 0.005
iterat

Instructions for updating:
Use standard file APIs to delete files with this prefix.


iteration: 60050 loss: 0.0022 lr: 0.02
iteration: 60100 loss: 0.0024 lr: 0.02
iteration: 60150 loss: 0.0023 lr: 0.02
iteration: 60200 loss: 0.0023 lr: 0.02
iteration: 60250 loss: 0.0027 lr: 0.02
iteration: 60300 loss: 0.0026 lr: 0.02
iteration: 60350 loss: 0.0024 lr: 0.02
iteration: 60400 loss: 0.0023 lr: 0.02
iteration: 60450 loss: 0.0024 lr: 0.02
iteration: 60500 loss: 0.0025 lr: 0.02
iteration: 60550 loss: 0.0024 lr: 0.02
iteration: 60600 loss: 0.0030 lr: 0.02
iteration: 60650 loss: 0.0026 lr: 0.02
iteration: 60700 loss: 0.0024 lr: 0.02
iteration: 60750 loss: 0.0024 lr: 0.02
iteration: 60800 loss: 0.0028 lr: 0.02
iteration: 60850 loss: 0.0027 lr: 0.02
iteration: 60900 loss: 0.0021 lr: 0.02
iteration: 60950 loss: 0.0027 lr: 0.02
iteration: 61000 loss: 0.0025 lr: 0.02
iteration: 61050 loss: 0.0026 lr: 0.02
iteration: 61100 loss: 0.0025 lr: 0.02
iteration: 61150 loss: 0.0030 lr: 0.02
iteration: 61200 loss: 0.0027 lr: 0.02
iteration: 61250 loss: 0.0025 lr: 0.02
iteration: 61300 loss: 0.

In [ ]:
deeplabcut.evaluate_network(path_config_file, plotting=True)

In [29]:
##UNCOMMENT 2 LINES TO MANUALLY LABEL WITH GUI
# %gui wx
# deeplabcut.label_frames(path_config_file)
deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

Creating images with labels by Phil.
They are stored in the following folder: E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-10-02\labeled-data\run1retpro1_90-35-4msCam1_labeled.
They are stored in the following folder: E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-10-02\labeled-data\run1retpro1_90-35-4msCam2_labeled.
They are stored in the following folder: E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-10-02\labeled-data\run3circles_90-35-4msCam1_labeled.
They are stored in the following folder: E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-10-02\labeled-data\run3circles_90-35-4msCam2_labeled.
They are stored in the following folder: E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-10-02\labeled-data\run4addabd_90-35-4msCam1_labeled.
They are stored in the following folder: E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-10-02\labeled-data\run4addabd_90-35-4msCam2_labeled.
They are stored in the following folder: E:\Use

#### 6. Convert XMALab exports to DeepLabCut format
No spaces in marker names, otherwise 2D export fails (more header columns than data)


In [ ]:
deeplabcut.analyze_videos(path_config_file,vids_to_analyze, videotype='.avi')

In [5]:
deeplabcut.create_labeled_video(path_config_file,vids_to_analyze)

Starting %  Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\run10flexextLAR_95-3-4ms ['Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run10flexextLAR_95-3-4ms\\run10flexextLAR_95-3-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run10flexextLAR_95-3-4ms\\run10flexextLAR_95-3-4msCam2.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run1retpro1_90-35-4ms\\run1retpro1_90-35-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run1retpro1_90-35-4ms\\run1retpro1_90-35-4msCam2.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run2retpro2_90-35-4ms\\run2retpro2_90-35-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.b

100%|████████████████████████████████████████████████████████████████████████████████| 800/800 [00:12<00:00, 63.12it/s]


Starting %  Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\run10flexextLAR_95-3-4ms ['Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run10flexextLAR_95-3-4ms\\run10flexextLAR_95-3-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run10flexextLAR_95-3-4ms\\run10flexextLAR_95-3-4msCam2.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run1retpro1_90-35-4ms\\run1retpro1_90-35-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run1retpro1_90-35-4ms\\run1retpro1_90-35-4msCam2.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run2retpro2_90-35-4ms\\run2retpro2_90-35-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.b

100%|████████████████████████████████████████████████████████████████████████████████| 800/800 [00:13<00:00, 58.94it/s]


Starting %  Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\run1retpro1_90-35-4ms ['Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run10flexextLAR_95-3-4ms\\run10flexextLAR_95-3-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run10flexextLAR_95-3-4ms\\run10flexextLAR_95-3-4msCam2.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run1retpro1_90-35-4ms\\run1retpro1_90-35-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run1retpro1_90-35-4ms\\run1retpro1_90-35-4msCam2.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run2retpro2_90-35-4ms\\run2retpro2_90-35-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.bice

...

100%|████████████████████████████████████████████████████████████████████████████████| 800/800 [00:13<00:00, 60.05it/s]


Starting %  Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\run9addabd_95-3-4ms ['Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run10flexextLAR_95-3-4ms\\run10flexextLAR_95-3-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run10flexextLAR_95-3-4ms\\run10flexextLAR_95-3-4msCam2.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run1retpro1_90-35-4ms\\run1retpro1_90-35-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run1retpro1_90-35-4ms\\run1retpro1_90-35-4msCam2.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run2retpro2_90-35-4ms\\run2retpro2_90-35-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps

100%|████████████████████████████████████████████████████████████████████████████████| 800/800 [00:13<00:00, 59.48it/s]


Starting %  Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\run9addabd_95-3-4ms ['Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run10flexextLAR_95-3-4ms\\run10flexextLAR_95-3-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run10flexextLAR_95-3-4ms\\run10flexextLAR_95-3-4msCam2.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run1retpro1_90-35-4ms\\run1retpro1_90-35-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run1retpro1_90-35-4ms\\run1retpro1_90-35-4msCam2.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run2retpro2_90-35-4ms\\run2retpro2_90-35-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps

100%|████████████████████████████████████████████████████████████████████████████████| 800/800 [00:13<00:00, 59.11it/s]


## Extract outlier frames [optional step]

This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. This step has many options, so please look at:

In [6]:
deeplabcut.extract_outlier_frames?

Signature: deeplabcut.extract_outlier_frames(config, videos, videotype='avi', shuffle=1, trainingsetindex=0, outlieralgorithm='jump', comparisonbodyparts='all', epsilon=20, p_bound=0.01, ARdegree=3, MAdegree=1, alpha=0.01, extractionalgorithm='kmeans', automatic=False, cluster_resizewidth=30, cluster_color=False, opencv=True, savelabeled=True, destfolder=None)
Docstring:
   Extracts the outlier frames in case, the predictions are not correct for a certain video from the cropped video running from
   start to stop as defined in config.yaml.

   Another crucial parameter in config.yaml is how many frames to extract 'numframes2extract'.

   Parameter
   ----------
   config : string
       Full path of the config.yaml file as a string.

   videos : list
       A list of strings containing the full paths to videos for analysis or a path to the directory, where all the videos with same extension are stored.

   videotype: string, optional
       Checks for the extension of the video in case

In [9]:
deeplabcut.extract_outlier_frames(path_config_file,vid_list) #pass a specific video

network parameters: DeepCut_resnet50_1_3_4_5_unenhancedSep30shuffle1_680000
Method  jump  found  279  putative outlier frames.
Do you want to proceed with extracting  20  of those?
If this list is very large, perhaps consider changing the paramters (start, stop, epsilon, comparisonbodyparts) or use a different method.


yes/no yes


Frames from video run1retpro1_90-35-4msCam1  already extracted (more will be added)!
Loading video...
Duration of video [s]:  26.666639999999997 , recorded @  30.00003000003 fps!
Overall # of frames:  800 with (cropped) frame dimensions: 
Kmeans-quantization based extracting of frames from 0.0  seconds to 26.67  seconds.
Extracting and downsampling... 279  frames from the video.


279it [00:10, 26.08it/s]


Kmeans clustering ... (this might take a while)
Let's select frames indices: [77, 501, 579, 314, 482, 28, 85, 534, 612, 552, 767, 564, 108, 733, 478, 469, 357, 548, 653, 558]
New video was added to the project! Use the function 'extract_frames' to select frames for labeling.
The outlier frames are extracted. They are stored in the subdirectory labeled-data\run1retpro1_90-35-4msCam1.
Once you extracted frames for all videos, use 'refine_labels' to manually correct the labels.
Method  jump  found  513  putative outlier frames.
Do you want to proceed with extracting  20  of those?
If this list is very large, perhaps consider changing the paramters (start, stop, epsilon, comparisonbodyparts) or use a different method.


...

Kmeans clustering ... (this might take a while)
Let's select frames indices: [27, 278, 9, 780, 482, 349, 567, 101, 86, 445, 729, 216, 385, 794, 545, 234, 284, 432, 710, 694]
New video was added to the project! Use the function 'extract_frames' to select frames for labeling.
The outlier frames are extracted. They are stored in the subdirectory labeled-data\run5flexextLAR_90-35-4msCam1.
Once you extracted frames for all videos, use 'refine_labels' to manually correct the labels.
Method  jump  found  615  putative outlier frames.
Do you want to proceed with extracting  20  of those?
If this list is very large, perhaps consider changing the paramters (start, stop, epsilon, comparisonbodyparts) or use a different method.


yes/no yes


Frames from video run5flexextLAR_90-35-4msCam2  already extracted (more will be added)!
Loading video...
Duration of video [s]:  26.666639999999997 , recorded @  30.00003000003 fps!
Overall # of frames:  800 with (cropped) frame dimensions: 
Kmeans-quantization based extracting of frames from 0.0  seconds to 26.67  seconds.
Extracting and downsampling... 615  frames from the video.


615it [00:17, 34.65it/s]


Kmeans clustering ... (this might take a while)
Let's select frames indices: [378, 462, 239, 423, 511, 710, 266, 343, 734, 546, 365, 340, 466, 623, 332, 702, 154, 190, 396, 407]
New video was added to the project! Use the function 'extract_frames' to select frames for labeling.
The outlier frames are extracted. They are stored in the subdirectory labeled-data\run5flexextLAR_90-35-4msCam2.
Once you extracted frames for all videos, use 'refine_labels' to manually correct the labels.


## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset. 

In [5]:
%gui wx
deeplabcut.refine_labels(path_config_file)

Windows
Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Windows
Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Windows
Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Windows
Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Windows
Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Closing... The refined labels are stored in a subdirectory under labeled-data. Use the function 'merge_datasets' to augment the training dataset, and then re-train a network using creat

**NOTE:** Afterwards, if you want to look at the adjusted frames, you can load them in the main GUI by running: ``deeplabcut.label_frames(path_config_file)``

(you can add a new "cell" below to add this code!)

#### Once all folders are relabeled, check the labels again! If you are not happy, adjust them in the main GUI:

``deeplabcut.label_frames(path_config_file)``

Check Labels:

``deeplabcut.check_labels(path_config_file)``

In [6]:
#NOW, merge this with your original data:

deeplabcut.merge_datasets(path_config_file)

Merged data sets and updated refinement iteration to 1.
Now you can create a new training set for the expanded annotated images (use create_training_dataset).


## Create a new iteration of training dataset [optional step]
Following the refinement of labels and appending them to the original dataset, this creates a new iteration of training dataset. This is automatically set in the config.yaml file, so let's get training!

In [8]:
deeplabcut.create_training_dataset(path_config_file)

E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\training-datasets\iteration-1\UnaugmentedDataSet_1_3_4_5_unenhancedSep30  already exists!
E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\dlc-models\iteration-1\1_3_4_5_unenhancedSep30-trainset95shuffle1  already exists!
E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\dlc-models\iteration-1\1_3_4_5_unenhancedSep30-trainset95shuffle1//train  already exists!
E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\dlc-models\iteration-1\1_3_4_5_unenhancedSep30-trainset95shuffle1//test  already exists!
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


In [11]:
deeplabcut.train_network(path_config_file, maxiters=200000, displayiters=100)

Config:
{'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16],
                [17],
                [18],
                [19],
                [20],
                [21],
                [22],
                [23],
                [24],
                [25],
                [26]],
 'all_joints_names': ['Body_ds1_crn',
                      'Body_ds2_int',
                      'Body_ds3_cdl',
                      'Body_vn1_crn',
                      'Body_vn2_int',
                      'Body_vn3_cdl',
                      'Body_acc',
                      'Scapula_acr',
                      'Scapula_spi',
                      'Scapula_vtb',
                      'Scapula_acc

Starting with standard pose-dataset loader.
INFO:tensorflow:Restoring parameters from C:\Users\LabAdmin\.conda\envs\dlc-windowsGPU\lib\site-packages\deeplabcut\pose_estimation_tensorflow\models\pretrained\resnet_v1_50.ckpt
Max_iters overwritten as 200000
Display_iters overwritten as 100
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'weigh_only_present_joints': False, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': 'E:\\Users\\Phil\\DeepLabCut\\dev\\1_3_4_5_unenhanced-Phil-2019-09-30\\dlc-models\\iteration-1\\1_3_4_5_unenhancedSep30-trainset95shuffle1\\train\\snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'mirror': False, 'crop_pad': 0, 'scorem

iteration: 100 loss: 0.0861 lr: 0.005
iteration: 200 loss: 0.0263 lr: 0.005
iteration: 300 loss: 0.0250 lr: 0.005
iteration: 400 loss: 0.0234 lr: 0.005
iteration: 500 loss: 0.0217 lr: 0.005
iteration: 600 loss: 0.0244 lr: 0.005
iteration: 700 loss: 0.0220 lr: 0.005
iteration: 800 loss: 0.0218 lr: 0.005
iteration: 900 loss: 0.0207 lr: 0.005
iteration: 1000 loss: 0.0201 lr: 0.005
iteration: 1100 loss: 0.0171 lr: 0.005
iteration: 1200 loss: 0.0177 lr: 0.005
iteration: 1300 loss: 0.0163 lr: 0.005
iteration: 1400 loss: 0.0161 lr: 0.005
iteration: 1500 loss: 0.0150 lr: 0.005
iteration: 1600 loss: 0.0146 lr: 0.005
iteration: 1700 loss: 0.0141 lr: 0.005
iteration: 1800 loss: 0.0160 lr: 0.005
iteration: 1900 loss: 0.0143 lr: 0.005
iteration: 2000 loss: 0.0137 lr: 0.005
iteration: 2100 loss: 0.0138 lr: 0.005
iteration: 2200 loss: 0.0138 lr: 0.005
iteration: 2300 loss: 0.0143 lr: 0.005
iteration: 2400 loss: 0.0137 lr: 0.005
iteration: 2500 loss: 0.0133 lr: 0.005
iteration: 2600 loss: 0.0135 lr: 0

The network is now trained and ready to evaluate. Use the function 'evaluate_network' to evaluate the network.


## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

THIS HAS MANY FUN OPTIONS! 

``deeplabcut.create_labeled_video(config, videos, videotype='avi', shuffle=1, trainingsetindex=0, filtered=False, save_frames=False, Frames2plot=None, delete=False, displayedbodyparts='all', codec='mp4v', outputframerate=None, destfolder=None, draw_skeleton=False, trailpoints=0, displaycropped=False)``

So please check:

In [4]:
deeplabcut.create_labeled_video?

In [ ]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [7]:
%matplotlib notebook #for making interactive plots.
deeplabcut.plot_trajectories(path_config_file,videofile_path)

UsageError: unrecognized arguments: #for making interactive plots.
